<a href="https://colab.research.google.com/github/ayamlearning/machine-learning-zoomcamp/blob/main/L3_Car_Price_prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
data = 'wget wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'
!wget $data

--2023-10-02 12:39:41--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2023-10-02 12:39:41--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2023-10-02 12:39:41--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data.csv.9’

data.csv.9          100%[===================>]   1.41M  --.-KB/s    in 0.01s   

2023-10-02 12:39:41 (115 MB/s) - ‘data.csv.9’ saved [1475504/1475504]

FINISHED --2023-10-02 12:39:41--
Total wall clock time: 0.07s
Downloaded: 1 files, 1.4M in 0.01s (115 MB

In [90]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [137]:
data = pd.read_csv('data.csv')
data.sample(5)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
3372,Mazda,CX-5,2016,regular unleaded,184.0,4.0,AUTOMATIC,all wheel drive,4.0,Crossover,Midsize,4dr SUV,30,24,586,29870
8540,Acura,RL,2011,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Luxury,Performance",Midsize,Sedan,24,17,204,51350
2552,Honda,Civic,2015,regular unleaded,110.0,4.0,AUTOMATIC,front wheel drive,4.0,Hybrid,Compact,Sedan,47,44,2202,25935
10202,Toyota,Tacoma,2016,regular unleaded,278.0,6.0,AUTOMATIC,rear wheel drive,4.0,NaN,Compact,Crew Cab Pickup,24,19,2031,28345
3371,Mazda,CX-5,2016,regular unleaded,184.0,4.0,AUTOMATIC,all wheel drive,4.0,Crossover,Midsize,4dr SUV,30,24,586,24895


In [138]:
len(data)

11914

## Initial data preparation

In [139]:
columns = ['Make', 'Model', 'Year', 'Engine HP','Engine Cylinders',
           'Transmission Type','Vehicle Style','highway MPG', 'city mpg']
df = data[columns]

In [140]:
df.head()

,Make,Model,Year,Engine HP,Engine Cylinders,Transmission Type,Vehicle Style,highway MPG,city mpg
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18


In [141]:
df.sample(5).T

,4792,10228,5432,8057,3456
Make,Ford,Toyota,Volkswagen,Dodge,Dodge
Model,Focus,Tacoma,Golf SportWagen,RAM 150,Dakota
Year,2017,2017,2015,1992,2009
Engine HP,160.0,278.0,150.0,180.0,210.0
Engine Cylinders,4.0,6.0,4.0,6.0,6.0
Transmission Type,AUTOMATED_MANUAL,AUTOMATIC,AUTOMATED_MANUAL,MANUAL,AUTOMATIC
Vehicle Style,4dr Hatchback,Extended Cab Pickup,Wagon,Regular Cab Pickup,Crew Cab Pickup
highway MPG,40,24,42,17,18
city mpg,27,19,31,14,14


In [142]:
df.dtypes

Make                  object
Model                 object
Year                   int64
Engine HP            float64
Engine Cylinders     float64
Transmission Type     object
Vehicle Style         object
highway MPG            int64
city mpg               int64
dtype: object

In [143]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df.columns

<ipython-input-143-6bf00c6cf40d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.lower().str.replace(' ', '_')


Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg'],
      dtype='object')

In [144]:
df.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
dtype: int64

In [145]:
df['engine_hp'] = df['engine_hp'].fillna(0)
df['engine_cylinders'] = df['engine_cylinders'].fillna(0)

<ipython-input-145-d1826c9363d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['engine_hp'] = df['engine_hp'].fillna(0)
<ipython-input-145-d1826c9363d8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['engine_cylinders'] = df['engine_cylinders'].fillna(0)


In [146]:
data.rename(columns={"MSRP": "price"},inplace= True)

## Exploratory data analysis

Question 1

In [147]:
df['transmission_type'].mode()

0    automatic
Name: transmission_type, dtype: object

Question 2

In [148]:
numerical = ['engine_hp','year','engine_cylinders','highway_mpg','city_mpg']

In [149]:
corr_data = df[numerical]
df_corr = pd.DataFrame(corr_data)

corr_matrix = df_corr.corr()
print(corr_matrix)

                  engine_hp      year  engine_cylinders  highway_mpg  city_mpg
engine_hp          1.000000  0.338714          0.774851    -0.415707 -0.424918
year               0.338714  1.000000         -0.040708     0.258240  0.198171
engine_cylinders   0.774851 -0.040708          1.000000    -0.614541 -0.587306
highway_mpg       -0.415707  0.258240         -0.614541     1.000000  0.886829
city_mpg          -0.424918  0.198171         -0.587306     0.886829  1.000000


In [150]:
# engine_hp and year = > 0.338714
# engine_hp and engine_cylinders =>  0.774851
# highway_mpg and engine_cylinders => -0.614541
# highway_mpg and city_mpg =>  0.886829

Make price binary

In [151]:
def greater_than_mean(price):
  return (price >= data['price'].mean()).astype('int')

In [152]:
df['above_average'] = data['price'].apply(greater_than_mean)
df.sample(5)

<ipython-input-152-d0f1eb6abf4c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['above_average'] = data['price'].apply(greater_than_mean)


,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,above_average
2795,chevrolet,colorado,2016,305.0,6.0,automatic,crew_cab_pickup,24,17,0
2843,chrysler,concorde,2003,200.0,6.0,automatic,sedan,27,19,0
10629,chevrolet,trailblazer,2008,285.0,6.0,automatic,4dr_suv,20,14,0
1813,volkswagen,beetle_convertible,2014,210.0,4.0,manual,convertible,31,23,0
10949,toyota,tundra,2015,381.0,8.0,automatic,regular_cab_pickup,17,13,0


In [153]:
df.sample(5).T

,11526,1072,5721,6338,10365
make,bmw,audi,chevrolet,mitsubishi,gmc
model,x6,a4,hhr,lancer,terrain
year,2015,2015,2011,2016,2017
engine_hp,300.0,220.0,155.0,168.0,182.0
engine_cylinders,6.0,4.0,4.0,4.0,4.0
transmission_type,automatic,automatic,manual,automatic,automatic
vehicle_style,4dr_suv,sedan,wagon,sedan,4dr_suv
highway_mpg,27,31,32,31,28
city_mpg,18,22,22,24,20
above_average,1,0,0,0,0


In [154]:
df.nunique()

make                  48
model                914
year                  28
engine_hp            357
engine_cylinders       9
transmission_type      5
vehicle_style         16
highway_mpg           59
city_mpg              69
above_average          2
dtype: int64

# Split the data

In [155]:
from sklearn.model_selection import train_test_split

In [156]:
df.shape

(11914, 10)

In [157]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
print(df_train_full.shape, df_test.shape)

(9531, 10) (2383, 10)


In [158]:
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)
print(df_train.shape, df_val.shape)

(7148, 10) (2383, 10)


In [159]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values

In [160]:
del df_train['above_average']
del df_val['above_average']

### Question 3

In [161]:
from sklearn.metrics import mutual_info_score

In [162]:
categorical = ["make", "model", "transmission_type", "vehicle_style"]

In [163]:
def calculate_mi(series):
    return mutual_info_score(series, y_train)

df_mi = df_train[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi.head())
display(df_mi.tail())

,MI
model,0.462344
make,0.239769
vehicle_style,0.084143
transmission_type,0.020958


,MI
model,0.462344
make,0.239769
vehicle_style,0.084143
transmission_type,0.020958


Question 4

In [164]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [165]:
dv = DictVectorizer(sparse=False)

In [166]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')
dv.fit(train_dict)
X_train = dv.transform(train_dict)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [167]:
def train_model(features):
  train_dict = df_train[features].to_dict(orient='records')

  dv = DictVectorizer(sparse=False)
  dv.fit(train_dict)
  X_train = dv.transform(train_dict)

  model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
  model.fit(X_train, y_train)

  val_dict = df_val[features].to_dict(orient='records')
  X_val = dv.transform(val_dict)

  y_pred = model.predict_proba(X_val)[:, 1]
  above_avg = y_pred > 0.5

  return (y_val == above_avg).mean()

In [168]:
accuracy = train_model(categorical + numerical)
print("Acuracy: ",round(accuracy,2))

Acuracy:  0.95


Question 5

In [169]:
features = categorical + numerical
global_accuracy = train_model(features)
print("Acuracy: ",global_accuracy)

Acuracy:  0.9471254720939991


In [178]:
features = categorical + numerical
temp =  ['year','engine_hp','transmission_type','city_mpg']
feature_dict = {}

for f in temp:
  temp_dict = {}
  features.remove(f)
  accuracy = train_model(features)
  temp_dict["accuracy"]=accuracy
  temp_dict["diff"]= global_accuracy - accuracy
  temp_dict["abs_diff"]= np.abs(global_accuracy - accuracy)

  features.append(f)
  feature_dict[f] = temp_dict

feature_dict

{'year': {'accuracy': 0.9483843894250944,
  'diff': -0.0012589173310952884,
  'abs_diff': 0.0012589173310952884},
 'engine_hp': {'accuracy': 0.9227864036928242,
  'diff': 0.024339068401174946,
  'abs_diff': 0.024339068401174946},
 'transmission_type': {'accuracy': 0.9454469156525388,
  'diff': 0.0016785564414603105,
  'abs_diff': 0.0016785564414603105},
 'city_mpg': {'accuracy': 0.9324381032312211,
  'diff': 0.014687368862777994,
  'abs_diff': 0.014687368862777994}}

Question 6

In [180]:
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,above_average
0,bmw,1_series_m,2011,335.0,6.0,manual,coupe,26,19,1
1,bmw,1_series,2011,300.0,6.0,manual,convertible,28,19,1
2,bmw,1_series,2011,300.0,6.0,manual,coupe,28,20,0
3,bmw,1_series,2011,230.0,6.0,manual,coupe,28,18,0
4,bmw,1_series,2011,230.0,6.0,manual,convertible,28,18,0


In [181]:
df_sample = df.copy()
del df_sample['above_average']

In [182]:
df_sample.sample(5)

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg
9197,suzuki,sidekick,1998,95.0,4.0,manual,convertible_suv,24,21
7748,chrysler,pt_cruiser,2010,150.0,4.0,automatic,wagon,24,19
1209,honda,accord_hybrid,2017,212.0,4.0,automatic,sedan,47,49
6472,land_rover,lr4,2016,340.0,6.0,automatic,4dr_suv,19,15
5182,pontiac,g6,2008,219.0,6.0,automatic,coupe,26,17


In [183]:
df_reg = pd.concat([df_sample,data.price],axis='columns')
df_reg.sample(5)

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
8827,mercedes-benz,s-class,2016,449.0,8.0,automatic,coupe,24,16,121550
4105,cadillac,escalade_esv,2015,420.0,8.0,automatic,4dr_suv,20,14,82570
10658,ford,transit_connect,2015,169.0,4.0,automatic,passenger_minivan,28,20,24710
8866,audi,s4,2014,333.0,6.0,automated_manual,sedan,28,18,55400
8933,audi,s7,2016,450.0,8.0,automated_manual,sedan,27,17,82900


In [184]:
df_train_full, df_test = train_test_split(df_reg, test_size=0.2, random_state=42)
print(df_train_full.shape, df_test.shape)

(9531, 10) (2383, 10)


In [185]:
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)
print(df_train.shape, df_val.shape)

(7148, 10) (2383, 10)


In [186]:
y_train = np.log1p(df_train.price.values)
y_val = np.log1p(df_val.price.values)

In [187]:
del df_train['price']
del df_val['price']

In [188]:
df_train.sample(5)

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg
11174,aston_martin,v8_vantage,2016,430.0,8.0,automated_manual,coupe,21,14
115,mazda,2,2012,100.0,4.0,automatic,4dr_hatchback,34,28
9614,chevrolet,silverado_1500,2017,285.0,6.0,automatic,regular_cab_pickup,22,17
5058,infiniti,fx,2011,303.0,6.0,automatic,4dr_suv,21,16
11230,hyundai,veloster,2015,201.0,4.0,automatic,2dr_hatchback,31,24


In [189]:
y_train[:5]

array([10.42228135, 10.17526888, 12.42118806, 10.12627104,  9.92700875])

In [190]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [191]:
def train_model(alp):
  train_dict = df_train.to_dict(orient='records')
  dv = DictVectorizer(sparse=False)
  dv.fit(train_dict)

  X_train = dv.transform(train_dict)

  model = Ridge(alpha=alp,solver='sag',random_state=42)
  model.fit(X_train, y_train)

  val_dict = df_val.to_dict(orient='records')
  X_val = dv.transform(val_dict)

  y_pred = model.predict(X_val)

  rmse = mean_squared_error(y_val, y_pred, squared = False)
  return rmse

In [193]:
alpha_pars = [0, 0.01, 0.1, 1, 10]

for a in alpha_pars:
  rmse = train_model(a)
  print("Alpha: ",a, "======>",round(rmse,3))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Alpha:  0 ======> 0.487


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Alpha:  0.01 ======> 0.487


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Alpha:  0.1 ======> 0.487


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Alpha:  1 ======> 0.487
Alpha:  10 ======> 0.487


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
